
# Intel® Geti™ SDK from Zero to Hero in 9 steps

Intel® Geti™ SDK is a Python package designed to interact with an Intel® Geti™ server via the REST API. It provides various functions for managing projects, downloading and uploading data, deploying projects for local inference, configuring projects and models, launching and monitoring training jobs, and media upload and prediction. Clone and install this repo 
https://github.com/openvinotoolkit/geti-sdk

| ![image-7.png](https://github.com/openvinotoolkit/geti-sdk/assets/76463150/43b1ded9-5ec2-4e62-a4aa-01bb974302dc) | 
|:--:| 
| *Intel® Geti™ Platform* |

|![image.png](https://github.com/openvinotoolkit/geti-sdk/assets/76463150/933896fb-90d1-4f20-aac0-24dc5e553ffa) | 
|:--:| 
| *What you can do with Intel® Geti™ SDK* |

Before running this notebook, please make sure you have the Intel Geti SDK installed in your local machine. If not, please follow [these instructions](https://github.com/openvinotoolkit/geti-sdk#installation).

## Ready for the 9 steps?
![image-2.png](https://github.com/openvinotoolkit/geti-sdk/assets/76463150/e37c090c-9cdf-448b-af4a-35b05ad1e8fe)

In this notebook, we will use the SDK to create a project on the Intel Geti graphics platform, upload videos, download the displays locally, and run the inference by viewing the results on this same notebook.

# Step 1: Connect with your Intel® Geti™ Instance

We will connect to the platform first, using the server details from the .env file. We will also create a ProjectClient for the server. If you have doubts, please take a look of the previous work you need to do before to [connect](https://github.com/openvinotoolkit/geti-sdk#connecting-to-the-intel-geti-platform) the SDK and the platform. 

In [1]:
# As usual we will connect to the platform first, using the server details from the .env file. We will also create a ProjectClient for the server
import os

from geti_sdk import Geti
from geti_sdk.rest_clients import ProjectClient
from geti_sdk.utils import get_server_details_from_env

geti_server_configuration = get_server_details_from_env(
    env_file_path=os.path.join("..", ".env")
)

geti = Geti(server_config=geti_server_configuration)

project_client = ProjectClient(session=geti.session, workspace_id=geti.workspace_id)

# Step 2: Create a project and upload a video for further annotations

We will create a new project from scratch and will upload a video using the SDK for further annotations into the Intel Geti Platform. We will create an object detection project for person, bike and card detection. 

| ![image-3.png](https://github.com/openvinotoolkit/geti-sdk/assets/76463150/6f338f36-014a-443a-987f-d672a75f908c) | 
|:--:| 
| *Video Frame from person-bike-car detection data* |

### Setting up the project client, video client and prediction client
For now, we will need two client objects: A `ProjectClient` to retrieve the project we want to upload to, and an `VideoClient` to be able to upload the video. We first set up the `ProjectClient`, since we will need to get the project we are interested in before we can initialize the another client.

In [2]:
from geti_sdk.rest_clients import ProjectClient, VideoClient

project_client = ProjectClient(
    session=geti.session, workspace_id=geti.workspace_id
)  # setting up the project client

projects = (
    project_client.list_projects()
)  # listing the projects in the Intel Geti Instance

#### Project creation
For project creation we will need: 1. Project Name, 2. Project Type, and 3. Properties for each project task. See this notebook for an extensive [explanation](https://github.com/openvinotoolkit/geti-sdk/blob/main/notebooks/001_create_project.ipynb). For this use case we will create an detection task.

In [3]:
# First set the project parameters. Feel free to experiment here!
PROJECT_NAME = "person-bike-car"  # the project we want to create
PROJECT_TYPE = "detection"  # the type of computer vision task to perform
LABELS = [["person", "car", "bike"]]  # The label names for the task

In [4]:
# Now, use the project client to create the project
project = project_client.get_or_create_project(
    project_name=PROJECT_NAME, project_type=PROJECT_TYPE, labels=LABELS
)

# Step 3: Uploading a video

We can upload a video directly from file using the `video_client.upload_video()` method. Before uploading, we can get a list of all videos in the project, so that we can verify that the image was uploaded successfully. With the project name specified, we can retrieve the project details from the project client and use the returned `Project` object to set up an `video_client` and `prediction_client` for this project.

In [5]:
project = project_client.get_project_by_name(PROJECT_NAME)

video_client = VideoClient(
    session=geti.session, workspace_id=geti.workspace_id, project=project
)

In [6]:
# videos in the project before uploading
videos = video_client.get_all_videos()
print(f"Project '{project.name}' contains {len(videos)} videos.")

Now, we will upload an example video from the SDK. Of course, you can replace the `VIDEO_PATH` with a path to one of your own videos as well.

In [7]:
from geti_sdk.demos import get_person_car_bike_video

# Get the path to the example video. This will download the video if it is not found on your disk yet
VIDEO_PATH = get_person_car_bike_video()

video = video_client.upload_video(video=VIDEO_PATH)

Let's fetch the list of videos again and see if it has changed

In [8]:
videos = video_client.get_all_videos()
print(f"Project '{project.name}' contains {len(videos)} videos.")

# Step 4: Creating annotations
Once you upload new image or video data, you should open the Intel Geti GUI and create annotations for it. The screenshot below shows an example of the annotator page within Geti.

| ![image.png](https://github.com/openvinotoolkit/geti-sdk/assets/76463150/d6c95554-5eec-474f-94a2-23ac6782786b) | 
|:--:| 
| *Annotations within the Intel® Geti™ Platform* |

Alternatively, if you have used the default 'person_car_bike' video that we provided, you can run the cell below to upload some pre-defined annotations for the video to the project. This saves you some time in annotating the frames.

In [9]:
from geti_sdk.annotation_readers import GetiAnnotationReader
from geti_sdk.rest_clients import AnnotationClient

annotation_reader = GetiAnnotationReader(
    os.path.join("data", "102_from_zero_to_hero", "annotations")
)
annotation_client = AnnotationClient(
    workspace_id=geti.workspace_id,
    session=geti.session,
    project=project,
    annotation_reader=annotation_reader,
)

annotation_client.upload_annotations_for_all_media()

# Step 5: Training the model
Once sufficient annotations have been made, the project is ready for training. Due to the incremental learning mechanism within the Intel Geti platform, training will happen automatically and frequently. Whenever sufficient new annotations have been created, the platform will start a training round. 

In the next part of the notebook we will deploy the model that was trained, so that we can use it locally to generate predictions. However, before doing so we need to make sure that the project has a model trained. The cell below ensures this: It will start training and monitor the training progress if no model is available yet.

In [ ]:
from geti_sdk.demos import ensure_trained_example_project

ensure_trained_example_project(geti=geti, project_name=PROJECT_NAME);

# Step 6: Download the model and save the deployment
When the model is ready you can download the deployment and run it locally or run the inference in the platform. In this example we will download the deployment and run it locally.

Once we are sure that the project has trained models for each task, we can create the deployment in the cell below.

In [ ]:
deployment = geti.deploy_project(project_name=PROJECT_NAME)

## Saving the deployment
When we create the deployment, the model data is saved to a temporary folder. We store the deployment for offline re-use later on by saving it: This will copy the model data from the temporary folder to the path we specify. If we want to run inference locally again, we can simply reload the deployment from the saved folder, without having to connect to the platform again.

In [ ]:
PATH_TO_DEPLOYMENT_FOLDER = os.path.join("deployments", PROJECT_NAME)

deployment.save(path_to_folder=PATH_TO_DEPLOYMENT_FOLDER)

## Preparing the models for inference
Now that the `deployment` is created and the models are saved to the local disk, we can load the models into memory to prepare them for inference. There you can select the device for running the inference, in OpenVINO we have different options. You can setup `CPU`, `GPU`, `AUTO` for [auto plugin](https://docs.openvino.ai/latest/openvino_docs_OV_UG_supported_plugins_AUTO.html), and `MULTI` for runnig the model in [multiple devices](https://docs.openvino.ai/latest/openvino_docs_OV_UG_Running_on_multiple_devices.html).

In [ ]:
deployment.load_inference_models(device="CPU")

# Step 7: Run the inference and digest new data into the Platform

We will run the inference locally and send some detection frames to the Intel Geti Platform, in order to annotate those and retrain a new model.

What happens if something new comes in your production system? Different acquisition conditions, lighting, camera, backgrounds. You can connect your production system with Intel Geti Platform in a flexible way through the Intel Geti SDK.

Note: For this use case we will send images back to the Intel Geti Platform when the number of detections per frame will be higher than 1.

## Setting up the image client
Previously, we had setup the `ProjectClient`, and the `VideoClient`. For connecting and sending some new frames back to the Platform we need to setup an `ImageClient` to be able to upload the image.

In [ ]:
from geti_sdk.rest_clients import ImageClient

image_client = ImageClient(
    session=geti.session, workspace_id=geti.workspace_id, project=project
)

## Preparing payload function for sending back frames to the Platform
In `utils\upload.py` we will find an `Uploader` class to perform multithreaded uploading to the Geti platform. The main purpose of this is to avoid any delay in the video visualization in the notebook, while still being able to upload frames on-the-go. 

In [ ]:
from utils import Uploader

uploader = Uploader(num_worker_threads=2, image_client=image_client)

## Main function for running the inference with video files or USB camera

This main function create a video player object to manage video files or USB cameras. By default we play the video in 30 FPS, and every single frame will be analyzed by the model. It also runs the inference using the Intel Geti SDK and create a queue of frames to be sent back to the Intel Geti platform.

Essentially, the code has four main components:

1. `VideoPlayer(source=source, flip=flip, fps=30, skip_first_frames=skip_first_frames)`: Custom video player to fulfill FPS requirements, you can set a webcam or video file, target FPS and output size, flip the video horizontally or skip first N frames.


2. `prediction = deployment.infer(frame)`: This generates a prediction for the image or frame. The prediction contains a list of methods and variables, such as `annotations` which contains information about boundung boxes, labels, confidence and color.


3. `uploader.add_data(input_image)`: uploader is a class to help us to create a separate thread for sending images back to the platform. This class is using image_client for that purposes.


4. `Visualizer(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR), prediction)`: this function helps us to have bounding boxes, labels and confidence over the actual frame for visualization purposes. 

In [ ]:
import collections
import time

import cv2
import numpy as np
from IPython import display
from utils import VideoPlayer

from geti_sdk.prediction_visualization.visualizer import Visualizer


# Main processing function to run object detection.
def run_object_detection(source=0, flip=False, use_popup=False, skip_first_frames=0):
    visualizer = Visualizer()
    player = None
    fps = 0
    try:
        # ===================1. Create a video player to play with target fps================
        player = VideoPlayer(
            source=source, flip=flip, fps=30, skip_first_frames=skip_first_frames
        )
        # Start capturing.
        player.start()
        if use_popup:
            title = "Press ESC to Exit"
            cv2.namedWindow(
                winname=title, flags=cv2.WINDOW_GUI_NORMAL | cv2.WINDOW_AUTOSIZE
            )

        processing_times = collections.deque()
        while True:
            # Grab the frame.
            frame = player.next()
            if frame is None:
                print("Source ended")
                break
            # If the frame is larger than full HD, reduce size to improve the performance.
            scale = 1280 / max(frame.shape)
            if scale < 1:
                frame = cv2.resize(
                    src=frame,
                    dsize=None,
                    fx=scale,
                    fy=scale,
                    interpolation=cv2.INTER_AREA,
                )
            input_image = frame.copy()

            # Measure processing time.

            start_time = time.time()

            # ==========================2. Using Geti SDK predictions========================

            # Get the results.
            prediction = deployment.infer(frame)
            stop_time = time.time()
            processing_times.append(stop_time - start_time)

            # ==========================3. Sending images back to the platform======================

            # if the prediction has more than one label send the image back to Intel Geti Platform
            if len(prediction.annotations) > 1:
                # image = image_client.upload_image(input_image)
                uploader.add_data(input_image)
                print(f"queue = {uploader.queue_length}")

            # ================4. Creating output with bounding boxes, labels, and confidence========
            output = visualizer.draw(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), prediction)
            output = cv2.cvtColor(output, cv2.COLOR_RGB2BGR)

            # Use processing times from last 200 frames.
            if len(processing_times) > 200:
                processing_times.popleft()

            _, f_width = frame.shape[:2]
            # Mean processing time [ms].
            processing_time = np.mean(processing_times) * 1000
            fps = 1000 / processing_time
            # print the FPS for your reference
            print(fps)

            # Use this workaround if there is flickering.
            if use_popup:
                cv2.imshow(winname=title, mat=output)
                key = cv2.waitKey(1)
                # escape = 27
                if key == 27:
                    break
            else:
                # Encode numpy array to jpg.
                _, encoded_img = cv2.imencode(
                    ext=".jpg", img=output, params=[cv2.IMWRITE_JPEG_QUALITY, 100]
                )
                # Create an IPython image.
                i = display.Image(data=encoded_img)
                # Display the image in this notebook.
                display.clear_output(wait=True)
                display.display(i)
    # ctrl-c
    except KeyboardInterrupt:
        print("Interrupted")
    # any different error
    except RuntimeError as e:
        print(e)
    finally:
        if player is not None:
            # Stop capturing.
            player.stop()
        if use_popup:
            cv2.destroyAllWindows()

# Step 8: Run the main function with video

Using the previous main function, we will run the inference in real time over this notebook and we will see the bounding boxes and the detection on a new video.

In [ ]:
# In this example, we run inference on the same video that we uploaded to the project. Of course `video_file`
# can be set to a different video for a more realistic scenario
video_file = VIDEO_PATH

run_object_detection(source=video_file, flip=False, use_popup=False)

This example showcases Intel® Geti™ SDK with a video file, but you can use live camera streams or so in a similar manner, just change the source argument in the previous function to make it equal the the camera source number.

# Step 9: Repeat Step 4, create/edit annotations in the platform
Once you upload new data to Geti, you should open the GUI and check, approve or edit the annotations. After accepting or editing a sufficient number of annotations, the platform will start a new round of model training. This training round takes your suggestions into account, in order to further improve the model.

When the model is ready you can download the deployment again and use it to obtain predictions, just like we did before. 


| ![image.png](https://github.com/openvinotoolkit/geti-sdk/assets/76463150/f65c8973-b151-47a7-b291-19bb3c3b8694) | 
|:--:| 
| *Interactive annotation with the Intel® Geti™ Platform* |

Alternatively, if you have used the default 'person_car_bike' video that we provided, you can run the cell below to upload some pre-defined annotations for the video to the project. This saves you some time in annotating the frames.